In [1]:
from utils_v1 import *


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates,
    PA, PADeforestationTime, PADistanceMap, PAMultipleDates,
    MTDeforestationTime,
    MTMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential

from src.trainer.evidential import TrainerEvidential

from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 



In [3]:

config = {
	"training": False,
	"save_probabilities": False,
	"loadInference": False,
	"addPastDeforestationInput": True,
	"inference_times": 1,
	"Uncertainty": {
		"annealing_step": 10*375/2 # 1875
	}
}




'''
if config["training"] == True:
	dates = [2017, 2018]
else:
	dates = [2018, 2019]
if config["training"] == True:
	dates = [2018, 2019]
else:
	dates = [2019, 2020]
'''
dates = [2018, 2019]
ic(dates)
dataset = PAMultipleDates(dates = dates, # dates = [2017, 2018, 2019]. [2018, 2019]. [2015, 2016, 2017, 2018]
	addPastDeforestationInput = config['addPastDeforestationInput'],
	borderBuffer = 2)	
# dataset = MTMultipleDates(dates = dates, # dates = [2016,2017,2018,2019]
# 	addPastDeforestationInput = addPastDeforestationInput,
# 	borderBuffer = 2)	
# dataset = MTDeforestationTime()	
# [2016, 2017, 2018]


if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerEvidential(
		dataset)
else:
	patchesHandler = PatchesHandler()



ic| dates: [2018, 2019]
ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [4]:
[10*375//1, 10*375//1.5, 10*375//2, 10*375//3, 10*375//4]


[3750, 2500.0, 1875, 1250, 937]

In [ ]:
# if config["training"] == True:
# experiment_values = [10*375//1, 10*375//1.5, 10*375//2, 10*375//3, 10*375//4]
# experiment_values = [10*375//3, 10*375//3, 10*375//3, 10*375//3, 10*375//3]
# experiment_values = [10*375//2, 10*375//2, 10*375//2, 10*375//2, 10*375//2, 10*375//2, 10*375//2]
experiment_values = [10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375]

experiment_values = [int(x) for x in experiment_values]
results = []

#exp_id_base = 148
# exp_id_base = 155
exp_id_base = 159


exp_ids = range(exp_id_base, exp_id_base+len(experiment_values))


In [ ]:
exp_ids

In [5]:


for experiment_value, exp_id in zip(experiment_values, exp_ids):
    config = {
        "training": True,
        "save_probabilities": False,
        "loadInference": False,
        "addPastDeforestationInput": True,
        "inference_times": 1,
        "Uncertainty": {
            "annealing_step": experiment_value# 10*375/2 # 1875
        }
    }

    trainer = TrainerEvidential(config, dataset, patchesHandler)

    trainer.defineExperiment(exp_id)

    trainer.setExperimentPath()
    trainer.createLogFolders()

    trainer.loadDataset()

    # %%
    if config["training"] == True:
        trainer.run()

    # %%
    result = trainer.run_predictor()
    results.append(result)


    with open('log.pkl', 'wb') as f:
        pickle.dump(results, f)

D:/Jorge/datasets/deforestation/Para/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Label where deforestation past years is actual date (2019) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

ic| self.predicted_unpad.shape: (17730, 9200)
    self.predicted.shape: (17730, 9200)
ic| predicted_test.shape: (81558000,)
ic| predicted_test.shape: (54236371,)
ic| np.unique(predicted, return_counts=True): (array([0.], dtype=float32), array([163116000], dtype=int64))
    np.unique(predicted_larger_than_min_area, return_counts=True): (array([0.], dtype=float32), array([163116000], dtype=int64))
ic| np.unique(ignored_polygons, return_counts=True): (array([0.], dtype=float32), array([163116000], dtype=int64))
c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
ic| self.f1: 0.0, self.precision: 0.0, self.recall: 0.0
ic| self.label_mask_val.shape: (16311600,)
ic| self.label_mask_val_valid.shape: (9450701,)
c:\Users\jchamorro\Anaconda3\env

(array([0., 1.], dtype=float32), array([161277492,   1838508], dtype=int64))


ic| self.label_mask.shape: (17730, 9200)
ic| self.mask_amazon_ts.shape: (17730, 9200)
ic| self.label_mask_test.shape: (81558000,)
ic| self.error_mask_test.shape: (54236371,)
ic| bounds: (0.0015, 0.9985)
c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\scipy\optimize\_minimize.py:783: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


0.38232011321635484
threshold 0.38232011321635484
(0,) (0,)
cm_correct []


IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
print(results)

[{'metrics': {'precision_L': array([0.95284808]), 'recall_L': array([0.92463037]), 'recall_Ltotal': array([0.66954444]), 'AA': array([0.03000392]), 'precision_H': array([0.7081642]), 'recall_H': array([0.45883842]), 'f1_L': array([0.93852717]), 'f1_H': array([0.55686755]), 'f1': 84.63}, 'metrics_audited': {'precision': array([0.96614136]), 'recall': array([0.94542325]), 'f1': array([0.95567003])}}, {'metrics': {'precision_L': array([0.70923349]), 'recall_L': array([0.94157394]), 'recall_Ltotal': array([0.81054622]), 'AA': array([0.02999964]), 'precision_H': array([0.21662722]), 'recall_H': array([0.34096309]), 'f1_L': array([0.80905351]), 'f1_H': array([0.26493246]), 'f1': 72.65}, 'metrics_audited': {'precision': array([0.74079538]), 'recall': array([0.9497044]), 'f1': array([0.83234158])}}, {'metrics': {'precision_L': array([0.90384691]), 'recall_L': array([0.93505106]), 'recall_Ltotal': array([0.71629424]), 'AA': array([0.02999986]), 'precision_H': array([0.54003029]), 'recall_H': ar

In [12]:
trainer.f1

0.0

In [11]:
print(trainer.f1)

0.0
